In [0]:
%run "/Workspace/Shared/Connection and Helper Function"

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
from pyspark.sql.functions import when, col, lit, current_date, year, coalesce

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   983  100   983    0     0  13608      0 --:--:-- --:--:-- --:--:-- 13652


OK
Adding Microsoft repository...


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    88  100    88    0     0   1728      0 --:--:-- --:--:-- --:--:--  1760


deb [arch=amd64,armhf,arm64] https://packages.microsoft.com/ubuntu/22.04/prod jammy mainUpdating package index again...
Hit:1 https://packages.microsoft.com/ubuntu/22.04/prod jammy InRelease
Hit:2 https://repos.azul.com/zulu/deb stable InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Reading package lists...


W: https://packages.microsoft.com/ubuntu/22.04/prod/dists/jammy/InRelease: Key is stored in legacy trusted.gpg keyring (/etc/apt/trusted.gpg), see the DEPRECATION section in apt-key(8) for details.
W: https://repos.azul.com/zulu/deb/dists/stable/InRelease: Key is stored in legacy trusted.gpg keyring (/etc/apt/trusted.gpg), see the DEPRECATION section in apt-key(8) for details.


Installing ODBC Driver 17...
Reading package lists...
Building dependency tree...
Reading state information...
msodbcsql17 is already the newest version (17.10.6.1-1).
0 upgraded, 0 newly installed, 0 to remove and 66 not upgraded.
Reading package lists...
Building dependency tree...
Reading state information...
unixodbc-dev is already the newest version (2.3.9-5ubuntu0.1).
0 upgraded, 0 newly installed, 0 to remove and 66 not upgraded.
Checking installed driver...
/.
/opt
/opt/microsoft
/opt/microsoft/msodbcsql17
/opt/microsoft/msodbcsql17/include
/opt/microsoft/msodbcsql17/include/msodbcsql.h
/opt/microsoft/msodbcsql17/share
/opt/microsoft/msodbcsql17/share/resources
/opt/microsoft/msodbcsql17/share/resources/en_US
/opt/microsoft/msodbcsql17/share/resources/en_US/msodbcsqlr17.rll
/opt/microsoft/msodbcsql17/etc
/opt/microsoft/msodbcsql17/etc/odbcinst.ini
/opt/microsoft/msodbcsql17/lib64
/opt/microsoft/msodbcsql17/lib64/libmsodbcsql-17.10.so.6.1
/usr
/usr/lib
/usr/share
/usr/share/doc


[ODBC Driver 17 for SQL Server]


In [0]:
load_date = dbutils.widgets.get('load_date')

In [0]:
transaction_raw = spark.read.format('parquet').load(f'/mnt/adlslanding/credit_transactions/{load_date}')

In [0]:
transaction_cleaned = transaction_raw.filter(col('TransactionID').isNotNull() & col('CustomerID').isNotNull() & col('CityID').isNotNull() & col('PaymentType').isNotNull() & col('TransactionDate').isNotNull()).withColumn('Amount',coalesce(col('Amount'),lit('0')).cast('float')).withColumn('Status',coalesce(col('Status'),lit('Failed')))

In [0]:
customers_table = "(SELECT * FROM CURATED.CUSTOMER WHERE ACTIVE_IND='Y') AS tmp"

customers_df = spark.read \
    .format("jdbc") \
    .option("url", jdbc_url) \
    .option("dbtable", customers_table) \
    .option("user", username) \
    .option("password", password) \
    .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
    .load()

In [0]:
city_distance_table = "(SELECT * FROM CURATED.CITY_DISTANCE) AS tmp"

city_distance_df = spark.read \
    .format("jdbc") \
    .option("url", jdbc_url) \
    .option("dbtable", city_distance_table) \
    .option("user", username) \
    .option("password", password) \
    .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
    .load()

In [0]:
transaction_with_customer_city = transaction_cleaned.join(customers_df, transaction_cleaned.CustomerID == customers_df.CUSTOMER_ID, 'left').select(transaction_cleaned.TransactionID, transaction_cleaned.CustomerID, transaction_cleaned.CityID, transaction_cleaned.Amount, transaction_cleaned.PaymentType, transaction_cleaned.Status, transaction_cleaned.TransactionDate, customers_df.CITY_ID).withColumnRenamed('CityID','TRANSACTION_CITY_ID').withColumnRenamed('CITY_ID','CUSTOMER_CITY_ID')

In [0]:
calculated_distance = transaction_with_customer_city.join(city_distance_df, (transaction_with_customer_city.TRANSACTION_CITY_ID == city_distance_df.CITY_ID_TO) & (transaction_with_customer_city.CUSTOMER_CITY_ID == city_distance_df.CITY_ID), 'left').withColumnRenamed('TransactionID','TRANSACTION_ID').withColumnRenamed('CustomerID','CUSTOMER_ID').withColumnRenamed('Amount','AMOUNT').withColumnRenamed('PaymentType','PAYMENT_TYPE').withColumnRenamed('Status','STATUS').withColumnRenamed('TransactionDate','TRANSACTION_DATE').withColumnRenamed('DISTANCE','TRANSACTION_DISTANCE').drop('CITY_ID','CITY_NAME','CITY_NAME_TO')

In [0]:
fraud_criticality = calculated_distance.withColumn('FRAUD_CRITICALITY',when(col('TRANSACTION_DISTANCE')==0,lit('NIL')).when((col('TRANSACTION_DISTANCE')>0) & (col('TRANSACTION_DISTANCE')<500),'LOW').when((col('TRANSACTION_DISTANCE')>=500) & (col('TRANSACTION_DISTANCE')<1000),'MEDIUM').when(col('TRANSACTION_DISTANCE')>=1000,'HIGH').otherwise('UNKNOWN')).withColumn('TRANSACTION_CITY_ID',col('TRANSACTION_CITY_ID').cast('int')).withColumn('CUSTOMER_CITY_ID',col('CUSTOMER_CITY_ID').cast('int'))

In [0]:
executeQuery(f"DELETE FROM CURATED.CREDIT_TRANSACTIONS WHERE TRANSACTION_DATE = '{load_date.replace('/','-')}'")

In [0]:
fraud_criticality.write.jdbc(url=jdbc_url, table="CURATED.CREDIT_TRANSACTIONS", mode="append", properties=connection_properties)

In [0]:
dbutils.notebook.exit(fraud_criticality.count())